- Read in the excel file from the utils folder
- replace the "D:/Annotation Stuff/" with "/Users/debbiesubocz/capstone/Annotation Stuff/"
- try to find the corresponding file name in: 
- move img files from fo + most recent annotation json to new directory
- upload the img + json combo to roboflow in a new roboflow project

# Creating the directory for annotations needed for "Minus" Iter 2

In [48]:
import os
import shutil
import pandas as pd

Specify the file locations
- "csv_file_path" is to the csv file created to map the orignal annotations (from 2022?) json files to the correct image files. Original annotations had multiple annotation .json files associated with a single image file. The file at csv_file_path is a table associating the most recent annotation json file with the correct image file.

- "fo_imgs_path" is the file location where the more recent image files are saved. .wav files were processed to create png files in Nov 2023. Slight differences in the color scheme of the images, but the .json annotations should theoretically translate to these newly processed images since the only difference is in the color

- "final_df_output" is where the dataframe should be saved after joining the image file names from the files in the "fo_imgs_path" directory and the files in the "csv_file_path" table.



In [49]:
og_annot_file_path = '/Users/debbiesubocz/GitReps/hydropho-natics/utils/unique_images_annotations.csv'
fo_imgs_path = "/Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLin/"

final_df_output = 'fo_image_to_og_json.csv'
roboflow_directory = "/Users/debbiesubocz/capstone/for_roboflow_upload"

## Create a data frame for the original annotation table

In [50]:
# Read the CSV file with file info related to the original annotations
df_og = pd.read_csv(og_annot_file_path)

df_og['json_file_path'] = df_og['json_file_path'].str.replace('D:/Annotation Stuff/', '/Users/debbiesubocz/capstone/Annotation Stuff/', regex=False)

print(df_og.columns)
print(df_og.head(3))

Index(['image_file_name', 'image_file_path', 'json_file_path'], dtype='object')
           image_file_name                                    image_file_path  \
0   20181204T100004-File-8  D:/Annotation Stuff/MLFigs_Labeled_Oct_26_Chri...   
1  20181204T113004-File-16  D:/Annotation Stuff/MLFigs_Labeled_Oct_26_Chri...   
2  20181204T113004-File-26  D:/Annotation Stuff/MLFigs_Labeled_Oct_26_Chri...   

                                      json_file_path  
0  /Users/debbiesubocz/capstone/Annotation Stuff/...  
1  /Users/debbiesubocz/capstone/Annotation Stuff/...  
2  /Users/debbiesubocz/capstone/Annotation Stuff/...  


## Create the data frame for the spectrogram image files processed in Nov 2023

In [51]:
# Specify the directory path
directory_path = fo_imgs_path

# List to store file names
full_file_paths = []
file_names = []

# Iterate over all files in the directory
for entry in os.listdir(directory_path):
    # Check if it's a file and not a directory
    # if os.path.isfile(os.path.join(directory_path, entry)):
    #     file_names.append(entry)

    full_path = os.path.join(directory_path, entry)
    # Check if it's a file and not a directory
    if os.path.isfile(full_path):
        full_file_paths.append(full_path)
        file_names.append(entry)

# Print the list of file names
# print(file_names[0])
        
df_fo = pd.DataFrame({'fo_full_file_path': full_file_paths, 'fo_file_name': file_names})
df_fo['fo_file_name'] = df_fo['fo_file_name'].str.replace('_20Hz.png', '', regex=False)

# Display the DataFrame
print(df_fo.columns)
print(df_fo)


Index(['fo_full_file_path', 'fo_file_name'], dtype='object')
                                       fo_full_file_path  \
0      /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
1      /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
2      /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
3      /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
4      /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
...                                                  ...   
39743  /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
39744  /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
39745  /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
39746  /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   
39747  /Volumes/MacFriendly/1Dec2018_28Feb2019/20HzLi...   

                    fo_file_name  
0         20181204T100004-File-0  
1        20181204T100004-File-10  
2        20181204T100004-File-11  
3        20181204T100004-File-12  
4        20181204T100004-File-13  
...     

## Perform the join operation and publish the results to csv file

In [52]:
# Perform the join
merged_df = pd.merge(df_og, df_fo, left_on='image_file_name', right_on='fo_file_name', how='left')

# Create the 'successful join' column
merged_df['successful join'] = merged_df['fo_file_name'].notnull()

# Select only the required columns
final_df = merged_df[['image_file_name', 'fo_file_name', 'json_file_path', 'fo_full_file_path', 'successful join']]

# Display the new DataFrame
#print(final_df)
unsuccessful_joins = (~final_df['successful join']).sum()

# Print the count of unsuccessful joins
print(f"Number of unsuccessful joins: {unsuccessful_joins}")

# Write DataFrame to CSV file
# final_df.to_csv(final_df_output, index=False)

# print(f"DataFrame saved to '{final_df_output}'")


Number of unsuccessful joins: 0


In [53]:
print(final_df.columns)

Index(['image_file_name', 'fo_file_name', 'json_file_path',
       'fo_full_file_path', 'successful join'],
      dtype='object')


## Move the files to a new directory

## Code to move files to new directory

In [54]:
# Directory where the files will be copied
new_directory = roboflow_directory

# Create the directory if it doesn't exist
if not os.path.exists(new_directory):
    os.makedirs(new_directory)


# Function to copy files
def copy_files(file_paths):
    for file_path in file_paths:
        if os.path.exists(file_path):
            shutil.copy(file_path, new_directory)


#file_paths_test = ["/Users/debbiesubocz/capstone/test_labeling/20181205T203004-File-10.png", "/Users/debbiesubocz/capstone/test_labeling/20181205T203004-File-10.json"]

copy_files(final_df['fo_full_file_path'])
copy_files(final_df['json_file_path'])

print("Files copied successfully.")

#copy_files(file_paths_test)

Files copied successfully.
